In [1]:
from aceai.agent import AgentBase, ToolExecutor
from aceai.llm import LLMService
from aceai.llm.openai import OpenAI
from ididi import Graph
from aceai.tools import spec, tool
from typing import Annotated

In [3]:
@tool
def add(a: Annotated[int, spec(description="The first number to add")], b: Annotated[int, spec(description="The second number to add")]) -> int:
    """
    Adds two numbers together.
    """
    print(f"Adding {a} and {b}")
    return a + b

In [5]:
from dotenv import dotenv_values

values = dotenv_values(".env")

def build_agent() -> AgentBase:
    openai_llm = OpenAI(api_key=values["OPENAI_API_KEY"], default_model="gpt-4", default_stream_model="gpt-4-turbo")
    graph = Graph()

    llm_service = LLMService(
        providers=[openai_llm],
        timeout_seconds=120,
    )
    executor = ToolExecutor(tools=[add], graph=graph)

    return AgentBase(
        prompt="You are a helpful assistant.",
        default_model="gpt-4",
        llm_service=llm_service,
        executor=executor,
    )

agent = build_agent()

In [6]:
await agent.handle("What is the sum of 12 and 30?")

Adding 12 and 30


BadRequestError: Error code: 400 - {'error': {'message': "Unknown parameter: 'input[2].tool_calls'.", 'type': 'invalid_request_error', 'param': 'input[2].tool_calls', 'code': 'unknown_parameter'}}